In [125]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter


In [126]:
def filesave(dataframe,name):
    dataframe.to_csv(name,sep="\t", encoding="utf-8",index=False)
def fileread(name):
    return pd.read_csv(name,sep="\t",encoding="utf-8")

In [127]:
test=fileread('test/chosun_dataframe.csv')

In [128]:
test

,title,content,date,section,url
0,"신한금융, 3분기 순익 1.6조… 역대 최대 실적",조용병 신한금융지주 회장이 고객 중심 경영을 강조하고 있다.신한금융이 올해 3분기(...,2022.10.25 13:52,경제,https://www.chosun.com/economy/economy_general...
1,자금시장 경색에… 한은 ‘베이비스텝’ 전망 나와,‘레고랜드 사태’가 뇌관이 된 자금 시장 경색이 쉽게 풀릴 기미를 보이지 않으면서 ...,2022.10.25 04:16,경제,https://www.chosun.com/economy/economy_general...
2,일본은행 두번째 복면개입? 엔·달러 환율 149엔대에서 갑자기 4엔 하락,지난 20일 스즈키 슌이치 일본 재무상이 취재진과 만나는 모습. /로이터 연합뉴스달...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
3,매달 10만원씩 6년 부었는데 이자 12만원? 낮아도 너무 낮은 금리에 ‘저축보험’...,은행 수신금리가 나날이 오르자 오래 전 저축보험을 가입했다가 중도해지에 나서는 사람...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
4,"작년 종부세 수정 요구 1481건, 1년새 80% 폭증… 절반이 받아들여졌다",지난해 납세자들이 종합부동산세(종부세)를 과도하게 냈다며 수정을 요구한 경정 청구가...,2022.10.25 03:00,경제,https://www.chosun.com/economy/economy_general...
...,...,...,...,...,...
1985,"정성호 “이재명, 부패지옥·청렴천국 달고 살아… 김용 개인비리일수도”",이재명 더불어민주당 대표는 24일 검찰이 서울 여의도 민주당사 내 민주연구원을 압수...,2022.10.25 07:17,정치,https://www.chosun.com/politics/politics_gener...
1986,"한동훈에 “화내는 모습 부적절” 지적한 김남국, 잠시 후 “이 사람아” 고성",24일 법제사법위원회 국정감사에서 화제가 된 장면은 한동훈 법무부 장관과 김의겸 더...,2022.10.25 10:25,정치,https://www.chosun.com/politics/politics_gener...
1987,“이게 6만원…먹다 남은 거 아닙니다” 속초 횟집 가격 논란,강원도 속초의 한 횟집에서 6만원 어치 참돔과 2만원 어치 개불을 포장해왔다며 네티...,2022.10.25 11:48,사회,https://www.chosun.com/national/national_gener...
1988,시정 연설·사전차담회 모두 거부한 野… 5년 전 文때는 달랐다,윤석열 대통령이 25일 서울 여의도 국회에서 2023년도 예산안 및 기금운용계획안 ...,2022.10.25 10:06,politics/assembly,https://www.chosun.com/politics/assembly/2022/...


In [178]:
test.drop_duplicates(inplace=True, ignore_index=True)

In [182]:
len(test)

1788

In [180]:
test['content'].nunique()

1749

In [218]:
def save_sentence_dataframe(data):
    idx=0
    sentences = []
    section_temp = []
    for i in range(len(data)):
        content = data['content'].iloc[idx]
        if "▲" in str(content):
            temp = str(content).split("▲")
            content=temp[0]
        splited = str(content).replace(". ","\n").split("\n")
        sentences += splited
        section_temp += [data['section'].iloc[idx]]*len(splited)
        idx+=1
    df = pd.DataFrame([sentences,section_temp]).T
    df.columns = ["sentence","section"]
    filesave(df, "donga_sentence_section.csv")

In [219]:
save_sentence_dataframe(test)

In [220]:
ttt=fileread("donga_sentence_section.csv")

In [210]:
def topic_counter_by_section(section,data):
    okt=Okt()
    parsed_data=[]
    sectiondata = data[data["section"]==section]
    for i in sectiondata['content']:
        if "▲" in str(i):
            temp = str(i).split("▲")
            i=temp[0]
        splited_list = str(i).replace(". ","\n").split("\n")
        if not splited_list:
            continue
        for sentence in splited_list:
            if len(sentence)<6:
                continue
            parsed_data += okt.pos(sentence, norm=True, stem=True)
    word_list = []
    
    stopword = ["없다","있다","재밌다","좋다","최고","같다","이","것","진짜","아니다","이렇다","그냥","하나","다시","많다","이다","최악",'완전',"안되다",'어떻다',"정말","보고",'정도','역시']
    for word,tag in parsed_data:
        if tag in ["Noun"] and len(word)>1 and word not in stopword:
            word_list.append(word)
    counts = Counter(word_list)
    return counts

In [232]:
def topic_counter_by_section_from_sentence_csv(section,data):
    okt=Okt()
    parsed_data=[]
    sectiondata = data[data["section"]==section]
    for sentence in sectiondata['sentence']:
        if len(str(sentence))<6:
            continue
        nouns = okt.nouns(str(sentence))
        ["없다","있다","재밌다","좋다","최고","같다","이","것","진짜","아니다","이렇다","그냥","하나","다시","많다","이다","최악",'완전',"안되다",'어떻다',"정말","보고",'정도','역시']
        for word in nouns:
            if len(word)>1 and word not in stopword:
                parsed_data.append(word)
    counts = Counter(parsed_data)
    return counts

In [212]:
def most_40topiclist_by_section(section,data):
    okt=Okt()
    parsed_data=[]
    sectiondata = data[data["section"]==section]
    for i in sectiondata["title"]:
        parsed_data+=okt.pos(i,norm=True,stem=True)

    word_list = []
    stopword = ["없다","있다","재밌다","좋다","최고","같다","이","것","진짜","아니다","이렇다","그냥","하나","다시","많다","이다","최악",'완전',"안되다",'어떻다',"정말","보고",'정도','역시']
    for word,tag in parsed_data:
        if tag in ["Noun"] and len(word)>1 and word not in stopword:
            word_list.append(word)


    counts = Counter(word_list)
    tags = counts.most_common(40)
    return tags

In [214]:
sections = test["section"].value_counts()

In [215]:
sections = list(sections[sections>21].keys())

In [216]:
sections

['사회', '연예', '국제', '정치', '경제', '문화', '스포츠']

In [135]:
word_count_df = pd.DataFrame(columns=['word','count','section'])

In [136]:
word_count_df

,word,count,section


In [137]:
# counter=topic_counter_by_section("경제",test)

KeyboardInterrupt: 

In [ ]:
# temp = pd.DataFrame([dict(counter)]).T
# temp['word'] = temp.index
# temp['section'] = "경제"
# temp.columns=['count','word','section']
# word_count_df = pd.concat([word_count_df,temp],ignore_index=True)

In [ ]:
# word_count_df

In [223]:
def make_word_count_df():
    df = pd.DataFrame(columns=['word','count','section'])
    for section in sections:
        print(section)
        counter=topic_counter_by_section(section,test)
        temp = pd.DataFrame([dict(counter)]).T
        temp['word'] = temp.index
        temp['section'] = section
        temp.columns=['count','word','section']
        df = pd.concat([df,temp],ignore_index=True)
    return df

In [224]:
def make_word_count_df2(data,section_list):
    df = pd.DataFrame(columns=['word','count','section'])
    for section in section_list:
        print(section)
        counter=topic_counter_by_section_from_sentence_csv(section,data)
        temp = pd.DataFrame([dict(counter)]).T
        temp['word'] = temp.index
        temp['section'] = section
        temp.columns=['count','word','section']
        df = pd.concat([df,temp],ignore_index=True)
    return df

In [141]:
# most_40topiclist_by_section("경제",test)

In [142]:
w = make_word_count_df()

사회
정치
연예
국제
경제
스포츠
문화


In [143]:
w

,word,count,section
0,송영,42,사회
1,한국,198,사회
2,유총,85,사회
3,연맹,240,사회
4,총재,68,사회
...,...,...,...
48548,청룡,1,문화
48549,도둑,1,문화
48550,암살,1,문화
48551,어콜,1,문화


In [144]:
filesave(w,"chosun_content_word_count.csv")

In [145]:
w['word'].nunique()

22723

In [152]:
w[w["section"]=="연예"].sort_values(by="count",ascending=False).head(15)

,word,count,section
14227,기자,381,연예
14236,공개,241,연예
14296,방송,241,연예
14241,모습,240,연예
14225,스포츠조선,203,연예
14400,사람,180,연예
15088,남편,176,연예
14233,사진,167,연예
14255,결혼,162,연예
14346,라며,160,연예


In [165]:
cnt = 0
datafilter = test[test['section']=="연예"]["content"].str.contains("방송")

285     True
286     True
287     True
288     True
289     True
        ... 
1771    True
1772    True
1773    True
1774    True
1775    True
Name: content, Length: 266, dtype: bool

In [242]:
t=test[(test['section']=="국제")&(test['content'].str.contains("러시아"))]

In [243]:
cnt=0
for i in t['content']:
    print(i)
    
    cnt+=1
    if cnt>6:
        break

드미트리 페스코프 러시아 크렘린궁 대변인. /타스 연합뉴스러시아 크렘린궁, 국방부, 외교부가 일제히 나서 이틀 째 우크라이나가 ‘더티밤(dirty bomb)’을 쓸 수 있다는 여론전을 이어가고 있다. 미국 등 서방 국가들은 “근거가 없는 허위 주장”이라고 일축했다.로이터통신에 따르면 24일(현지 시각) 드미트리 페스코프 크렘린궁 대변인은 기자들과 전화 회의에서 “러시아가 전달한 정보를 서방이 믿지 않는다고 해서 더티밤의 위협이 없어지는 것은 아니다”라고 했다. 더티밤은 재래식 폭탄에 방사성 물질을 채운 방사능 무기로, 핵폭발과 같은 파괴력은 없지만 광범위한 지역을 방사능으로 오염시킬 수 있어 핵테러 폭발물로 분류된다.전날 세르게이 쇼이구 러시아 국방장관이 미국, 영국, 프랑스, 튀르키예(터키) 등 국방장관과의 통화에서 우크라이나가 더티밤을 전장에서 사용할 수 있다고 주장한 것을 반복한 것이다.이고르 키릴로프 러시아 국방부 화생방전 방어사령관/EPA 연합뉴스국방부는 전날 주장을 더욱 구체화했다. 러시아 국방부 화생방전 방어사령관 이고르 키릴로프는 이날 “우크라이나는 더티밤을 제조할 과학적, 기술적 역량이 있고 이를 사용할 동기도 갖고 있다”고 주장했다. 이어 “우크라이나 대통령실이 관련 기술을 전수 받고자 영국과 접촉했고, 더티밤에 쓸 방사성 물질을 체르노빌 원자력 발전소 등에 보관하고 있다”며 더티밤 개발이 마지막 단계라고 했다.또 더티밤 사용시 유럽 전역이 영향을 받을 것이라면서 “우크라이나는 러시아가 자국에서 대량 살상무기를 사용했다고 비난해 세계적으로 반(反) 러시아 캠페인을 일으키려는 것”이라고 했다. 그러면서 “국방부는 예상되는 도발에 대응할 준비가 됐다. 군대가 방사능 오염 상황에서도 임무를 수행할 준비가 됐다”며 이를 임박한 위협으로 묘사했다.세르게이 라브로프 러시아 외무장관도 이날 기자들과 만나 “더티밤을 만드는 우크라이나 내 과학 시설에 대한 정보를 갖고 있다”며 “적절한 경로로 거듭 확인한 내용이다. 근거 없는 추측이 아니다”라고 강조했다. 

In [228]:
w2 = make_word_count_df2(ttt,['사회', '연예', '국제', '정치', '경제', '문화', '스포츠'])

사회
연예
국제
정치
경제
문화
스포츠


In [229]:
w2

,word,count,section
0,송영,4,사회
1,무,14,사회
2,한국,122,사회
3,유총,9,사회
4,연맹,31,사회
...,...,...,...
52047,타쿠야,1,스포츠
52048,추첸첸,1,스포츠
52049,격상,1,스포츠
52050,선도,1,스포츠


In [231]:
w2.sort_values(by="count", ascending=False)

,word,count,section
22451,것,1229,정치
22353,의원,943,정치
22407,고,928,정치
14431,것,893,국제
103,것,892,사회
...,...,...,...
35963,휴대,1,경제
35962,송화,1,경제
35961,특,1,경제
35960,국제우편,1,경제


In [234]:
w2['word_length']=[len(i) for i in w2['word']]

In [237]:
w2 = w2[w2['word_length']>1]

In [238]:
w2.drop(['word_length'],axis=1,inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14012\1103842021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w2.drop(['word_length'],axis=1,inplace=True)


In [241]:
w2.sort_values(by="count", ascending=False)

,word,count,section
22353,의원,943,정치
14871,러시아,776,국제
22352,민주당,665,정치
23209,대표,664,정치
14879,우크라이나,652,국제
...,...,...,...
14234,대중문화,1,연예
14235,평론가,1,연예
37516,콜레스테롤,1,문화
14236,고유,1,연예


In [244]:
filesave(w2,"chosun_content_word_count.csv")